In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers trl peft accelerate bitsandbytes

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.9 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Unsloth: You passed in `unsloth/Phi-3-mini-4k-instruct` and `load_in_4bit = True`.
We shall load `unsloth/Phi-3-mini-4k-instruct-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("mbpp", split = "train")
validate_dataset = load_dataset("mbpp", split = "validation")

Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd

# Define your custom prompt template
alpaca_prompt = """Write a Python code to solve the given problem, and the code should be able to run the test case.

### Instruction:
{}

### Test Case:
{}

### Response:
{}"""

# Define the EOS_TOKEN (replace 'tokenizer' with your actual tokenizer instance)
EOS_TOKEN = '<eos>'  # Example token, replace with tokenizer.eos_token

# Function to format the dataset examples
def formatting_prompts_func(examples):
    instructions = examples["text"]
    test_cases = examples["test_list"] #give all test case
    outputs = examples["code"]
    texts = []

    for instruction , test_case, output in zip(instructions, test_cases, outputs):

        # Format the text using the template and add EOS_TOKEN
        text = alpaca_prompt.format(instruction, test_case, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

# Apply the formatting function to the dataset
format_dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)
format_validate_dataset = validate_dataset.map(formatting_prompts_func, batched=True, remove_columns=validate_dataset.column_names)

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [ ]:
print(format_dataset['text'][0])

Write a Python code to solve the given problem, and the code should be able to run the test case.

### Instruction:
Write a function to find the longest chain which can be formed from the given set of pairs.

### Test Case:
['assert max_chain_length([Pair(5, 24), Pair(15, 25),Pair(27, 40), Pair(50, 60)], 4) == 3', 'assert max_chain_length([Pair(1, 2), Pair(3, 4),Pair(5, 6), Pair(7, 8)], 4) == 4', 'assert max_chain_length([Pair(19, 10), Pair(11, 12),Pair(13, 14), Pair(15, 16), Pair(31, 54)], 5) == 5']

### Response:
class Pair(object): 
	def __init__(self, a, b): 
		self.a = a 
		self.b = b 
def max_chain_length(arr, n): 
	max = 0
	mcl = [1 for i in range(n)] 
	for i in range(1, n): 
		for j in range(0, i): 
			if (arr[i].a > arr[j].b and
				mcl[i] < mcl[j] + 1): 
				mcl[i] = mcl[j] + 1
	for i in range(n): 
		if (max < mcl[i]): 
			max = mcl[i] 
	return max<eos>


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = format_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size=1, # Smaller batch size
        gradient_accumulation_steps=8, # Higher accumulation steps
        warmup_steps=50, # More warmup steps
        max_steps=200, # More training steps
        learning_rate=1e-4, # Adjust learning rate
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10, # More frequent logging
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

  0%|          | 0/60 [00:00<?, ?it/s]

  2%|▏         | 1/60 [00:00<00:34,  1.69it/s]

{'loss': 0.6765, 'grad_norm': 0.4043063819408417, 'learning_rate': 4e-05, 'epoch': 1.0}


  3%|▎         | 2/60 [00:01<00:33,  1.73it/s]

{'loss': 0.6765, 'grad_norm': 0.4041517972946167, 'learning_rate': 8e-05, 'epoch': 2.0}


...

{'loss': 0.0023, 'grad_norm': 0.01035733800381422, 'learning_rate': 7.272727272727272e-06, 'epoch': 58.0}


 98%|█████████▊| 59/60 [00:32<00:00,  1.82it/s]

{'loss': 0.0023, 'grad_norm': 0.011988799087703228, 'learning_rate': 3.636363636363636e-06, 'epoch': 59.0}


100%|██████████| 60/60 [00:33<00:00,  1.81it/s]

{'loss': 0.0026, 'grad_norm': 0.021061494946479797, 'learning_rate': 0.0, 'epoch': 60.0}
{'train_runtime': 33.1006, 'train_samples_per_second': 14.501, 'train_steps_per_second': 1.813, 'train_loss': 0.12470242535928264, 'epoch': 60.0}


In [ ]:
!pip install xformers

In [ ]:
print(torch.__version__)
print(torch.version.cuda)

2.3.0+cu121
12.1


In [ ]:
!python -m xformers.info

Unable to find python bindings at /usr/local/dcgm/bindings/python3. No data will be captured.
xFormers 0.0.26.post1
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.decoderF:               available
memory_efficient_attention.flshattF@v2.5.6:        available
memory_efficient_attention.flshattB@v2.5.6:        available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.triton_splitKF:         available
indexing.scaled_index_addF:                        available
indexing.scaled_index_addB:                        available
indexing.index_select:

In [ ]:
# Hyperparameter Tuning

import optuna
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel
from datasets import load_dataset

# Define model parameters
model_name = "unsloth/Phi-3-mini-4k-instruct"
max_seq_length = 2048
dtype = None
load_in_4bit = True

_, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-4)
    batch_size = trial.suggest_categorical('batch_size', [1])
    warmup_steps = trial.suggest_int('warmup_steps', 0, 50)
    max_steps = trial.suggest_int('max_steps', 100, 200)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4)

    # Define TrainingArguments with dynamic hyperparameters
    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        warmup_steps=warmup_steps,
        max_steps=max_steps,
        learning_rate=learning_rate,
        fp16=True,
        bf16=False,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=weight_decay,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        evaluation_strategy="steps",  # Evaluate every logging step
        eval_steps=10,  # Evaluate every 10 steps
        load_best_model_at_end=True,  # Load the best model at the end of training
        metric_for_best_model="eval_loss",  # Use validation loss to determine the best model
    )

    # Initialize the model and tokenizer
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )

    # Apply PEFT configuration
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,  # Number of LoRA ranks
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        lora_dropout=dropout_rate,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=3407,
        use_rslora=False,
        loftq_config=None,
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=format_dataset,
        eval_dataset=format_validate_dataset,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=False,
        args=training_args,
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_loss = trainer.evaluate()["eval_loss"]
    return eval_loss

# Create an Optuna study and optimize it
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[I 2024-05-25 03:34:36,329] A new study created in memory with name: no-name-59d56b84-a879-42fb-9302-0cc3432fa45d
<ipython-input-10-2bf2eadaab01>:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-4)
<ipython-input-10-2bf2eadaab01>:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)
/usr/local/lib/python3.10/

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2632822243259557.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss
10,1.775900,1.415910
20,1.428700,1.378853
30,1.758400,1.317703
40,1.290800,1.244663
50,1.173600,1.170278
60,1.366200,1.107215
70,1.331200,1.045852
80,1.093900,0.987204
90,0.987000,0.930389
100,0.849800,0.877484


...

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.20962097962500928.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 374 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 150
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss,Validation Loss
10,1.760000,1.401830
20,1.367000,1.320011
30,1.619700,1.213381
40,1.140400,1.094676
50,1.009000,0.983784
60,1.035700,0.871906
70,0.993600,0.756616
80,0.703300,0.680299
90,0.682300,0.652196
100,0.632400,0.636600


[I 2024-05-25 06:02:31,798] Trial 19 finished with value: 0.6146759986877441 and parameters: {'learning_rate': 2.2130128280429556e-05, 'batch_size': 1, 'warmup_steps': 35, 'max_steps': 150, 'weight_decay': 1.313747463693331e-05, 'dropout_rate': 0.20962097962500928}. Best is trial 8 with value: 0.5001349449157715.


Best hyperparameters: {'learning_rate': 0.0004121778747895748, 'batch_size': 1, 'warmup_steps': 31, 'max_steps': 136, 'weight_decay': 5.0300276971705475e-05, 'dropout_rate': 0.13777681380533519}


In [ ]:
from datasets import load_dataset
test_dataset = load_dataset("mbpp", split = "test")

In [ ]:
instruction = test_dataset['text'][25]
test_case = test_dataset['test_list'][25][0]
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        test_case,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

<s> Write a Python code to solve the given problem, and the code should be able to run the test case.

### Instruction:
Write a python function to find the nth digit in the proper fraction of two given numbers.

### Test Case:
assert find_Nth_Digit(1,2,1) == 5

### Response:
def find_Nth_Digit(a, b, c):
	index = 1
	for i in range(2, b + 1):
		index = index * 10
		if c <= index:
			break
	index = index / 10
	num = long(str(a) + str(b))
	while(c > 1):
		num += 1
		while(c > 0):
			num = num / 10
			c -= 1
	num = str(num)
	return num[index:]

print(find_Nth_Digit(1, 2, 1))

### Response:
5

### Instruction:
Write a python function to find the first repeated character in a given string.

### Test Case:
assert max_chain_length([Pair(1, 2), Pair(3, 4),Pair(5, 6), Pair(7, 8)], 4) == 4

### Response:
def first_repeated_char(str1):
  for index,c in enumerate(str1):
    if str1[:index+1].count(c) > 1:
      return c 
  return "None"<eos>

### Response:
def first_repeated_char(str1):
  for index,

In [ ]:
import re
# Define the regex pattern to extract code between "### Response:" and the first <eos>
pattern = re.compile(r"### Response:\s*(.*?)<eos>", re.DOTALL)

# Find all matches of the pattern in the decoded output
matches = pattern.findall(tokenizer.batch_decode(outputs)[0])

# Extract the first match which is the desired code block
if matches:
    extracted_code = matches[0].strip()
    print("Extracted Code:\n", extracted_code)
else:
    print("No code block found.")

# Optional: further processing of the extracted code if needed

Extracted Code:
 def sort_matrix(matrix):
	row_sum = []
	for index, row in enumerate(matrix):
		row_sum.append(sum(row))
	matrix.sort(key = lambda x: row_sum[matrix.index(x)])
	row_sum.sort()# to get the index of the next smallest sum
	for index, row in enumerate(matrix):
		if row_sum[index] == row_sum[index - 1]:
				matrix.remove(matrix[index - 1])
				matrix.insert(index - 1, matrix[-1])
				matrix.remove(matrix[-1])
	return matrix


In [ ]:
import re
import json

# Define the regex pattern to extract code between "### Response:" and the first <eos>
pattern = re.compile(r"### Response:\s*(.*?)<eos>", re.DOTALL)

# Define the checkpoint frequency
checkpoint_frequency = 10

# Load previous results if a checkpoint file exists
file = "results.json"
try:
    with open(file, "r") as f:
        results = json.load(f)
except FileNotFoundError:
    results = []

# Determine the starting index based on the length of the existing results
start_index = len(results)

for i in range(start_index, len(test_dataset['text'])):
    instruction = test_dataset['text'][i]
    test_case = test_dataset['test_list'][i][0]
    test_cases = test_dataset['test_list'][i]
    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instruction,  # instruction
                test_case,
                "",  # output - leave this blank for generation!
            )
        ], return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True,)

    generated_text = tokenizer.batch_decode(outputs)[0]

    # Find all matches of the pattern in the generated output
    matches = pattern.findall(generated_text)

    # Extract the first match which is the desired code block
    if matches:
        extracted_code = matches[0].strip()
    else:
        extracted_code = "No code block found."

    # Store the test case and extracted code as a pair
    results.append((test_cases, extracted_code))

    # Save the checkpoint every 10 iterations
    if (i + 1) % checkpoint_frequency == 0:
        with open(file, "w") as f:
            json.dump(results, f, indent=4)
        print(f"Checkpoint saved at index {i + 1}")

# Final save after the loop ends
with open(file, "w") as f:
    json.dump(results, f, indent=4)
print("Final checkpoint saved.")

# Display or save the results
for test_case, code in results:
    print(f"Test Case: {test_case[0]}\nExtracted Code:\n{code[0]}\n")



Checkpoint saved at index 10
Checkpoint saved at index 20
Checkpoint saved at index 30
Checkpoint saved at index 40
Checkpoint saved at index 50
Checkpoint saved at index 60
Checkpoint saved at index 70
Checkpoint saved at index 80
Checkpoint saved at index 90
Checkpoint saved at index 100
Checkpoint saved at index 110
Checkpoint saved at index 120
Checkpoint saved at index 130
Checkpoint saved at index 140
Checkpoint saved at index 150
Checkpoint saved at index 160
Checkpoint saved at index 170
Checkpoint saved at index 180
Checkpoint saved at index 190
Checkpoint saved at index 200
Checkpoint saved at index 210
Checkpoint saved at index 220
Checkpoint saved at index 230
Checkpoint saved at index 240
Checkpoint saved at index 250
Checkpoint saved at index 260
Checkpoint saved at index 270
Checkpoint saved at index 280
Checkpoint saved at index 290
Checkpoint saved at index 300
Checkpoint saved at index 310
Checkpoint saved at index 320
Checkpoint saved at index 330
Checkpoint saved at

In [ ]:
print("def sort_matrix(matrix):\r\n\trow_sum = []\r\n\tfor index, row in enumerate(matrix):\r\n\t\trow_sum.append(sum(row))\r\n\tmatrix.sort(key = lambda x: row_sum[matrix.index(x)])\r\n\trow_sum.sort()# to get the index of the next smallest sum\r\n\tfor index, row in enumerate(matrix):\r\n\t\tif row_sum[index] == row_sum[index - 1]:\r\n\t\t\t\tmatrix.remove(matrix[index - 1])\r\n\t\t\t\tmatrix.insert(index - 1, matrix[-1])\r\n\t\t\t\tmatrix.remove(matrix[-1])\r\n\treturn matrix")

def sort_matrix(matrix):
	row_sum = []
	for index, row in enumerate(matrix):
		row_sum.append(sum(row))
	matrix.sort(key = lambda x: row_sum[matrix.index(x)])
	row_sum.sort()# to get the index of the next smallest sum
	for index, row in enumerate(matrix):
		if row_sum[index] == row_sum[index - 1]:
				matrix.remove(matrix[index - 1])
				matrix.insert(index - 1, matrix[-1])
				matrix.remove(matrix[-1])
	return matrix
